In [1]:
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import classification_report,confusion_matrix
from sklearn import tree , datasets
import os

In [2]:
%load_ext autoreload
%autoreload 2

In [3]:
# data source path
data_path = r'./data'
final_data_path = r'./final_data'



In [4]:
# data read and rename columns 
birth_col = ['cow_number','confinement date','Date of dried milk','calf number1','calf number2','cow weight','login date'
            ,'Calculated tire','tire','difficulty level of confinement','calf size','calf gender','farm number']

report_col = ['ID','year','month','farm number','cow_number','father_cow_number','mother_cow_number','birthday','tire'
              ,'lactation date','milk support','last confinement date','sampling date','age of month','inspection date'
             ,'final breeding date','final breeding semen','breeding times','before confinement date','first breeding date'
             ,'first breeding semen']

birth_df = pd.read_csv(os.path.join(data_path , 'birth.csv') , names = birth_col,  skiprows=1)
report_df = pd.read_csv(os.path.join(data_path , 'report.csv') , names = report_col , skiprows=1)

C:\ProgramData\Anaconda3\lib\site-packages\IPython\core\interactiveshell.py:2785: DtypeWarning: Columns (6) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [5]:
len(report_col)

21

In [6]:
print('birth table : ')
print ("Dataset Shape:: ", birth_df.shape)
birth_df.head()

birth table : 
Dataset Shape::  (3761, 13)


,cow_number,confinement date,Date of dried milk,calf number1,calf number2,cow weight,login date,Calculated tire,tire,difficulty level of confinement,calf size,calf gender,farm number
0,87121677,2013/1/4 00:00,NaN,NaN,NaN,NaN,2013/1/25 00:00,True,2,1.0,NaN,NaN,B
1,96126940,2013/1/2 00:00,NaN,NaN,NaN,522.0,2013/1/25 00:00,True,2,2.0,NaN,NaN,B
2,95181708,2013/1/14 00:00,2014/1/3 00:00,2188113,NaN,670.0,2013/1/25 00:00,True,5,1.0,NaN,NaN,C
3,97127665,2013/1/9 00:00,2014/3/12 00:00,NaN,NaN,NaN,2013/1/25 00:00,True,3,2.0,NaN,NaN,B
4,97182634,2013/1/9 00:00,2014/3/25 00:00,NaN,NaN,630.0,2013/1/25 00:00,True,4,1.0,NaN,NaN,C


In [7]:
print('report table : ')
print ("Dataset Shape:: ", report_df.shape)
report_df.head()

report table : 
Dataset Shape::  (37517, 21)


,ID,year,month,farm number,cow_number,father_cow_number,mother_cow_number,birthday,tire,lactation date,...,last confinement date,sampling date,age of month,inspection date,final breeding date,final breeding semen,breeding times,before confinement date,first breeding date,first breeding semen
0,1,2013,3,A,52612,507H7173,96040545,2011/1/1 00:00,1,78.0,...,2013/1/1 00:00,2013/3/20 00:00,26,2013/3/21 00:00,2015/5/5 00:00,507H11621,1,NaN,2013/3/5 00:00,501H8784
1,2,2013,4,A,52612,507H7173,96040545,2011/1/1 00:00,1,107.0,...,2013/1/1 00:00,2013/4/18 00:00,27,2013/4/19 00:00,2015/5/5 00:00,507H11621,1,NaN,2013/3/5 00:00,501H8784
2,3,2013,5,A,52612,507H7173,96040545,2011/1/1 00:00,1,140.0,...,2013/1/1 00:00,2013/5/21 00:00,28,2013/5/22 00:00,2015/5/5 00:00,507H11621,2,NaN,2013/3/5 00:00,501H8784
3,4,2013,6,A,52612,507H7173,96040545,2011/1/1 00:00,1,175.0,...,2013/1/1 00:00,2013/6/25 00:00,29,2013/6/26 00:00,2015/5/5 00:00,507H11621,2,NaN,2013/3/5 00:00,501H8784
4,5,2013,7,A,52612,507H7173,96040545,2011/1/1 00:00,1,210.0,...,2013/1/1 00:00,2013/7/30 00:00,30,2013/7/31 00:00,2015/5/5 00:00,507H11621,2,NaN,2013/3/5 00:00,501H8784


In [8]:
# change string to int 

def replace_string_int(x , items):
    for i,j in items:
        if x == j :
            return str(i) 

def change_col(col_list,report_df):
    for col in col_list:
        farm_number_list = list(set(report_df[col]))
        turple_fram_number = [i for i in enumerate(farm_number_list)]
        #print(turple_fram_number)
        report_df[col] = report_df[col].apply(lambda x : replace_string_int(x , turple_fram_number))
        
    return report_df
    
change_col_list = ['farm number','final breeding semen','first breeding semen','father_cow_number','mother_cow_number']
report_df = change_col(change_col_list , report_df)




In [9]:
temp_df = report_df.drop(['birthday','lactation date','last confinement date','sampling date','inspection date'
                   ,'final breeding date','before confinement date','first breeding date'],axis=1)
temp_df = temp_df.dropna()
x = temp_df.drop('milk support',axis=1)
y = temp_df['milk support']

x_train, x_test, y_train, y_test = train_test_split( x, y, test_size = 0.2, random_state = 100)
print('x train shape {}'.format(x_train.shape))
print('x test shape {}'.format(x_test.shape))
print('y train shape {}'.format(y_train.shape))
print('y test shape {}'.format(y_test.shape))

x train shape (23229, 12)
x test shape (5808, 12)
y train shape (23229,)
y test shape (5808,)


In [10]:
from sklearn import linear_model
from sklearn.metrics import mean_squared_error as mse
reg = linear_model.LinearRegression()
reg.fit(x_train, y_train)
y_pred = reg.predict(x_test)

print('Linear Regression with MSE is {} score'.format(mse(y_pred ,y_test)))


Linear Regression with MSE is 51.334231490074785 score


In [11]:
# Bayesian Ridge Regression

reg = linear_model.BayesianRidge()
reg.fit(x_train, y_train)
y_pred = reg.predict(x_test)
print('Bayesian Ridge Regression with MSE is {} score'.format(mse(y_pred ,y_test)))

Bayesian Ridge Regression with MSE is 51.33317582485689 score


In [12]:
# Polynomial + linear model using pipeline
from sklearn.preprocessing import PolynomialFeatures
from sklearn.pipeline import Pipeline
model = Pipeline([('poly', PolynomialFeatures(degree=3)),
                   ('linear', linear_model.LinearRegression(fit_intercept=False))])
model.fit(x_train, y_train)
y_pred = model.predict(x_test)
print('Polynomial degree=3 + linear model using pipeline with MSE is {} score'.format(mse(y_pred ,y_test)))



Polynomial degree=3 + linear model using pipeline with MSE is 5676.9677162034595 score


In [13]:
# Support Vector Machines
from sklearn import svm

model = svm.SVR()
model.fit(x_train,y_train)
y_pred = model.predict(x_test)
print('Support Vector Machines Regression with MSE is {} score'.format(mse(y_pred ,y_test)))



Support Vector Machines Regression with MSE is 74.21419954290843 score


In [14]:
# Stochastic Gradient Descent Regression

model = linear_model.SGDRegressor(max_iter=1000, tol=1e-4)
model.fit(x_train,y_train)
y_pred = model.predict(x_test)
print('Stochastic Gradient Descent Regression with MSE is {} score'.format(mse(y_pred ,y_test)))



Stochastic Gradient Descent Regression with MSE is 6.286152168205721e+46 score


In [15]:
# Decision Tree Regression
from sklearn import tree
model = tree.DecisionTreeRegressor()
model.fit(x_train,y_train)
y_pred = model.predict(x_test)
print('Decision Tree Regression  with MSE is {} score'.format(mse(y_pred ,y_test)))

Decision Tree Regression  with MSE is 48.11710055096419 score


In [16]:
# Decision Tree Regression + Bagging Regressor
from sklearn import tree
from sklearn.ensemble import BaggingRegressor

model = BaggingRegressor(tree.DecisionTreeRegressor())
model.fit(x_train,y_train)
y_pred = model.predict(x_test)
print('Bagging Regressor + Decision Tree Regression  with MSE is {} score'.format(mse(y_pred ,y_test)))


Bagging Regressor + Decision Tree Regression  with MSE is 28.82585404614325 score


In [17]:
# RandomForestRegressor
from sklearn import tree
from sklearn.ensemble import RandomForestRegressor

model = RandomForestRegressor(max_depth=2, random_state=0)
model.fit(x_train,y_train)
y_pred = model.predict(x_test)
print('RandomForestRegressor  with MSE is {} score'.format(mse(y_pred ,y_test)))


RandomForestRegressor  with MSE is 55.45966234303219 score


In [18]:
# DecisionTreeRegressor + AdaBoostRegressor
from sklearn import tree
from sklearn.ensemble import AdaBoostRegressor

model = AdaBoostRegressor(tree.DecisionTreeRegressor(max_depth=4),n_estimators=1000, random_state=np.random.RandomState(1))
model.fit(x_train,y_train)
y_pred = model.predict(x_test)
print('DecisionTreeRegressor + AdaBoostRegressor  with MSE is {} score'.format(mse(y_pred ,y_test)))


DecisionTreeRegressor + AdaBoostRegressor  with MSE is 50.32202462098741 score


In [19]:
# GradientBoostingRegressor GBDT
from sklearn import tree
from sklearn.ensemble import GradientBoostingRegressor

model = GradientBoostingRegressor(n_estimators=100, learning_rate=0.1, max_depth=1, random_state=0, loss='ls')
model.fit(x_train,y_train)
y_pred = model.predict(x_test)
print('GradientBoostingRegressor  with MSE is {} score'.format(mse(y_pred ,y_test)))


GradientBoostingRegressor  with MSE is 49.65248777060311 score


In [22]:
# Voting Regressor
from sklearn import tree
from sklearn.ensemble import VotingRegressor,GradientBoostingRegressor,RandomForestRegressor

reg1 = GradientBoostingRegressor(random_state=1, n_estimators=10)
reg2 = RandomForestRegressor(random_state=1, n_estimators=10)
reg3 = linear_model.LinearRegression()


model = VotingRegressor(estimators=[('gb', reg1), ('rf', reg2), ('lr', reg3)])
model.fit(x_train,y_train)
y_pred = model.predict(x_test)
print('Voting Regressor  with MSE is {} score'.format(mse(y_pred ,y_test)))


Voting Regressor  with MSE is 38.09006638839522 score


In [26]:
# StackingRegressor
from sklearn import tree
from sklearn.ensemble import GradientBoostingRegressor , StackingRegressor ,RandomForestRegressor
from sklearn.svm import SVR

estimators = [('ridge', linear_model.RidgeCV()),
             ('lasso', linear_model.LassoCV(random_state=42)),
              ('svr', SVR(C=1, gamma=1e-6))]
final_layer = StackingRegressor(estimators=[('rf', RandomForestRegressor(random_state=42)),
                ('gbrt', GradientBoostingRegressor(random_state=42))],final_estimator=linear_model.RidgeCV())


multi_layer_regressor  = StackingRegressor(estimators=estimators , final_estimator = final_layer)
multi_layer_regressor.fit(x_train,y_train)
y_pred = multi_layer_regressor.predict(x_test)
print('Voting Regressor  with MSE is {} score'.format(mse(y_pred ,y_test)))


Voting Regressor  with MSE is 73.21674551982785 score
